In [2]:
import vanna 
from vanna.ollama import Ollama
from vanna.vannadb import VannaDB_VectorStore
import os
from dotenv import load_dotenv

In [3]:
# Connecting to vanna, we are using the defaul LLM (GPT 3.5) and Vector store (pgvector) provided by vanna
load_dotenv()
vanna_api_key = os.getenv("vanna_api_key")
vanna_model_name = "vannagetsomecoffee"

In [4]:
class CustomLLMVanna(VannaDB_VectorStore, Ollama):
    def __init__(self, config=None):
        MY_VANNA_MODEL = vanna_model_name# Your model name from https://vanna.ai/account/profile
        VannaDB_VectorStore.__init__(self, vanna_model=MY_VANNA_MODEL, vanna_api_key=vanna_api_key, config=config)
        Ollama.__init__(self, config=config)

vn = CustomLLMVanna(config={'model': 'llama3.1'})

In [5]:
# Since we are using a model that has already been trained with the db details we won't have to train it again

vn.connect_to_sqlite("chinook.db")

In [9]:
# Correct way of using vanna.ask()

vn.ask(question="What are the top 10 albums by sale and who are the artits of those albums", print_results=True, auto_train=False, visualize=False) 

SQL Prompt: [{'role': 'system', 'content': 'You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE "albums"\r\n(\r\n    [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Title] NVARCHAR(160)  NOT NULL,\r\n    [ArtistId] INTEGER  NOT NULL,\r\n    FOREIGN KEY ([ArtistId]) REFERENCES "artists" ([ArtistId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\r\n)\n\nCREATE TABLE "artists"\r\n(\r\n    [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Name] NVARCHAR(120)\r\n)\n\nCREATE TABLE "tracks"\r\n(\r\n    [TrackId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Name] NVARCHAR(200)  NOT NULL,\r\n    [AlbumId] INTEGER,\r\n    [MediaTypeId] INTEGER  NOT NULL,\r\n    [GenreId] INTEGER,\r\n    [Composer] NVARCHAR(220),\r\n    [Milliseconds] INTEGER  NOT NULL,\r\n    [Bytes] INTEGER,\r\n    [Uni

In [6]:
def askVanna(question):
    return vn.ask(question=question, print_results=True, auto_train=False, visualize=False) 


In [7]:
askVanna("Which customers are from brazil?")

SQL Prompt: [{'role': 'system', 'content': 'You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE "customers"\r\n(\r\n    [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [FirstName] NVARCHAR(40)  NOT NULL,\r\n    [LastName] NVARCHAR(20)  NOT NULL,\r\n    [Company] NVARCHAR(80),\r\n    [Address] NVARCHAR(70),\r\n    [City] NVARCHAR(40),\r\n    [State] NVARCHAR(40),\r\n    [Country] NVARCHAR(40),\r\n    [PostalCode] NVARCHAR(10),\r\n    [Phone] NVARCHAR(24),\r\n    [Fax] NVARCHAR(24),\r\n    [Email] NVARCHAR(60)  NOT NULL,\r\n    [SupportRepId] INTEGER,\r\n    FOREIGN KEY ([SupportRepId]) REFERENCES "employees" ([EmployeeId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\r\n)\n\nCREATE TABLE "invoices"\r\n(\r\n    [InvoiceId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [CustomerId] INTEGER  N

In [9]:
vn.run_sql("SELECT DISTINCT [FirstName], [LastName] FROM customers WHERE [Country] = 'Brazil'")

,FirstName,LastName
0,Luís,Gonçalves
1,Eduardo,Martins
2,Alexandre,Rocha
3,Roberto,Almeida
4,Fernanda,Ramos
